## Resnet 34 and inception v3

### ??? Test resnet with image size 224 ???

In [2]:
import os
import sys
import glob
import shutil
import random
import pickle
import numpy as np
from PIL import Image
import time
import copy
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.utils.data
from torchvision.models import resnet34, inception_v3
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler


# Add the src directory for functions
src_dir = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'src')
print(src_dir)
sys.path.append(src_dir)

# import my functions:
from functions import*

# Set it to use GPU1
torch.cuda.set_device(0)
print(torch.cuda.is_available())
print(torch.cuda.current_device())

/home/rene/code/idc/src
True
0


## Training ResNet34 on 10% sample
* Upsample the image to 197x197

In [ ]:
PATH = '/media/rene/Data/data/idc/sample'
num_workers = 6
batch_size=64

# Is 197 the min size for resnet? this is so ineffecient, the images are 50x50
dataloaders, dataset_sizes = make_batch_gen(PATH, batch_size, num_workers, valid_name='valid', size=197)

model_list = [resnet34]
model_name = ['resnet34']

epochs = 12
save_path = '/media/rene/Data/data/idc/sample/models'

for idx, model_arch in enumerate(model_list):
    model_ft = model_arch(pretrained=False)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()

    criterion = nn.CrossEntropyLoss()
    optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

    best_acc, model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                     epochs, dataloaders, dataset_sizes)
    torch.save(model_ft.state_dict(), os.path.join(save_path, model_name[idx]+'_'+str(idx)))

Epoch 0/11
----------
train Loss: 0.3949 Acc: 0.8375
valid Loss: 0.3705 Acc: 0.8349
Epoch 1/11
----------
train Loss: 0.3532 Acc: 0.8480
valid Loss: 0.4576 Acc: 0.7857
Epoch 2/11
----------
train Loss: 0.3447 Acc: 0.8565
valid Loss: 0.3519 Acc: 0.8465
Epoch 3/11
----------
train Loss: 0.3270 Acc: 0.8610
valid Loss: 0.3355 Acc: 0.8532
Epoch 4/11
----------
train Loss: 0.3225 Acc: 0.8596
valid Loss: 0.3374 Acc: 0.8520
Epoch 5/11
----------
train Loss: 0.3220 Acc: 0.8619
valid Loss: 0.3327 Acc: 0.8563
Epoch 6/11
----------
train Loss: 0.3173 Acc: 0.8635
valid Loss: 0.3271 Acc: 0.8578
Epoch 7/11
----------
train Loss: 0.3159 Acc: 0.8646
valid Loss: 0.3290 Acc: 0.8546
Epoch 8/11
----------
train Loss: 0.3149 Acc: 0.8660
valid Loss: 0.3265 Acc: 0.8578
Epoch 9/11
----------
train Loss: 0.3141 Acc: 0.8664
valid Loss: 0.3309 Acc: 0.8542
Epoch 10/11
----------
train Loss: 0.3125 Acc: 0.8670
valid Loss: 0.3276 Acc: 0.8558
Epoch 11/11
----------


In [ ]:
PATH = '/media/rene/Data/data/idc'
num_workers = 6
batch_size=64

# Is 197 the min size for resnet? this is so ineffecient, the images are 50x50
dataloaders, dataset_sizes = make_batch_gen(PATH, batch_size, num_workers, valid_name='valid', size=197)

model_list = [resnet34, resnet34, resnet34, resnet34, resnet34, resnet34]
model_name = ['resnet34', 'resnet34', 'resnet34', 'resnet34', 'resnet34', 'resnet34']

epochs = 10
save_path = '/media/rene/Data/data/idc/models'

for idx, model_arch in enumerate(model_list):
    model_ft = model_arch(pretrained=False)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()

    criterion = nn.CrossEntropyLoss()
    optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

    best_acc, model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                     epochs, dataloaders, dataset_sizes)
    torch.save(model_ft.state_dict(), os.path.join(save_path, model_name[idx]+'_'+str(idx)))

Epoch 0/9
----------
train Loss: 0.3556 Acc: 0.8466
valid Loss: 0.3326 Acc: 0.8620
Epoch 1/9
----------


## Results

In [5]:
model_arch = resnet34
model_name = 'resnet34_0'

save_path = '/media/rene/Data/data/idc/sample/models'
PATH = '/media/rene/Data/data/idc'
num_workers = 4
batch_size=64
dataloaders_valid, dataset_sizes_valid = make_batch_gen(PATH, batch_size, num_workers, valid_name='valid', size=197)
dataloaders_test, dataset_sizes_test = make_batch_gen(PATH, batch_size, num_workers, valid_name='test', size=197)

criterion = nn.CrossEntropyLoss()

# get the proper model architecture
model_ft = model_arch(pretrained=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

# load the saved weights
model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name)))
print('Sample Model Validation: ')
valid_loss, valid_acc = eval_model(model_ft, dataloaders_valid['valid'], dataset_sizes_valid['valid'], criterion)

print('Sample Model Test: ', model_name)
test_loss, test_acc = eval_model(model_ft, dataloaders_test['test'], dataset_sizes_test['test'], criterion)

Sample Model Validation: 
Loss: 0.3140 Acc: 0.8655
Sample Model Test:  resnet34_0
Loss: 0.3169 Acc: 0.8628


In [6]:
model_list = [resnet34, resnet34, resnet34, resnet34, resnet34, resnet34]
model_name = ['resnet34_0', 'resnet34_1', 'resnet34_2', 'resnet34_3', 'resnet34_4', 'resnet34_5']  

save_path = '/media/rene/Data/data/idc/models'
PATH = '/media/rene/Data/data/idc'
num_workers = 4
batch_size=64
dataloaders_valid, dataset_sizes_valid = make_batch_gen(PATH, batch_size, num_workers, valid_name='valid', size=197)
dataloaders_test, dataset_sizes_test = make_batch_gen(PATH, batch_size, num_workers, valid_name='test', size=197)

criterion = nn.CrossEntropyLoss()

results = {}

for idx, model_arch in enumerate(model_list):
    # get the proper model architecture
    model_ft = model_arch(pretrained=False)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()
    
    # load the saved weights
    model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name[idx])))
    print('Validation: ', model_name[idx])
    valid_loss, valid_acc = eval_model(model_ft, dataloaders_valid['valid'], dataset_sizes_valid['valid'], criterion)
    
    print('Test: ', model_name[idx])
    test_loss, test_acc = eval_model(model_ft, dataloaders_test['test'], dataset_sizes_test['test'], criterion)
    results[model_name[idx]] = [valid_acc, test_acc]
    
f = open(os.path.join(save_path, "resnet34_all6.pkl"),"wb")
pickle.dump(results,f)
f.close()

Validation:  resnet34_0
Loss: 0.2428 Acc: 0.8993
Test:  resnet34_0
Loss: 0.2463 Acc: 0.8979
Validation:  resnet34_1
Loss: 0.2497 Acc: 0.8951
Test:  resnet34_1
Loss: 0.2530 Acc: 0.8943
Validation:  resnet34_2
Loss: 0.2461 Acc: 0.8969
Test:  resnet34_2
Loss: 0.2486 Acc: 0.8971
Validation:  resnet34_3
Loss: 0.2496 Acc: 0.8958
Test:  resnet34_3
Loss: 0.2524 Acc: 0.8945
Validation:  resnet34_4
Loss: 0.2463 Acc: 0.8962
Test:  resnet34_4
Loss: 0.2496 Acc: 0.8960
Validation:  resnet34_5
Loss: 0.2486 Acc: 0.8973
Test:  resnet34_5
Loss: 0.2505 Acc: 0.8966


In [7]:
import operator

results2 = {key:value[0] for (key,value) in results.items()}
sorted_x = sorted(results2.items(), key=operator.itemgetter(1))
print(sorted_x)

[('resnet34_1', 0.8950731461240962), ('resnet34_3', 0.895841841024286), ('resnet34_4', 0.896178145043119), ('resnet34_2', 0.8969468399433087), ('resnet34_5', 0.8973311873934037), ('resnet34_0', 0.8993009680751399)]


### Sensitivity/Specificity

In [8]:
model_arch = resnet34
model_name = 'resnet34_2'

save_path = '/home/rene/data/idc/models'
PATH = '/home/rene/data/idc'
num_workers = 4
batch_size=64
dataloaders, dataset_sizes = make_batch_gen(PATH, batch_size, num_workers, valid_name='valid', test_name='test', size=197)

# get the proper model architecture
model_ft = model_arch(pretrained=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

# load the saved weights
model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name)))

In [9]:
model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name)))

all_labels, all_preds = get_preds(model_ft, dataloaders['valid'], dataset_sizes['valid'])
metrics = get_metrics_bin(all_labels, all_preds)
print(f'Validation sensitivity: {metrics["TPR"]}, specificity: {metrics["TNR"]}, Acc: {metrics["ACC"]}')

all_labels, all_preds = get_preds(model_ft, dataloaders['test'], dataset_sizes['test'])
metrics = get_metrics_bin(all_labels, all_preds)
print(f'Test sensitivity: {metrics["TPR"]}, specificity: {metrics["TNR"]}, Acc: {metrics["ACC"]}')

Validation sensitivity: 0.9213245732774274, specificity: 0.8318342151675485, Acc: 0.8969468399433087
Test sensitivity: 0.922548242135871, specificity: 0.8292124945006599, Acc: 0.8970669485214634


# Inception v3

In [3]:
PATH = '/media/rene/Data/data/idc'
num_workers = 6
batch_size=32
epochs = 10
save_path = '/media/rene/Data/data/idc/models'

dataloaders, dataset_sizes = make_batch_gen(PATH, batch_size, num_workers, valid_name='valid', test_name='test', size=299)
model_name = 'inception_v3'

model_ft = inception_v3(pretrained=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

In [ ]:
best_acc, model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                                 epochs, dataloaders, dataset_sizes)
torch.save(model_ft.state_dict(), os.path.join(save_path, model_name))

Epoch 0/9
----------
train Loss: 0.7370 Acc: 0.8444
valid Loss: 0.3178 Acc: 0.8643
Epoch 1/9
----------
train Loss: 0.6558 Acc: 0.8603
valid Loss: 0.3149 Acc: 0.8735
Epoch 2/9
----------
train Loss: 0.6186 Acc: 0.8676
valid Loss: 0.2857 Acc: 0.8805
Epoch 3/9
----------
train Loss: 0.5653 Acc: 0.8794
valid Loss: 0.2660 Acc: 0.8876
Epoch 4/9
----------


In [3]:
model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name)))
valid_loss, valid_acc = eval_model(model_ft, dataloaders['valid'], dataset_sizes['valid'], criterion)
print('Validation: ', valid_loss, valid_acc)

print('Test: ', model_name)
test_loss, test_acc = eval_model(model_ft, dataloaders['test'], dataset_sizes['test'], criterion)
print('Test: ', test_loss, test_acc)

Loss: 0.2526 Acc: 0.8956
Validation:  0.2526020526853055 0.8956016238679767
Test:  inception_v3
Loss: 0.2560 Acc: 0.8919
Test:  0.255982373306663 0.8918782579451825


## Sensitivity/Specificity

In [4]:
model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name)))

all_labels, all_preds = get_preds(model_ft, dataloaders['valid'], dataset_sizes['valid'], criterion)
metrics = get_metrics_bin(all_labels, all_preds)
print(f'Validation sensitivity: {metrics["TPR"]}, specificity: {metrics["TNR"]}, Acc: {metrics["ACC"]}')

all_labels, all_preds = get_preds(model_ft, dataloaders['test'], dataset_sizes['test'], criterion)
metrics = get_metrics_bin(all_labels, all_preds)
print(f'Test sensitivity: {metrics["TPR"]}, specificity: {metrics["TNR"]}, Acc: {metrics["ACC"]}')

Validation sensitivity: 0.9138956606533398, specificity: 0.8437960235640648, Acc: 0.8956016238679767
Test sensitivity: 0.9138381201044387, specificity: 0.8306488306488307, Acc: 0.8918782579451825
